### +DownLoad

In [1]:
! pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 66.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 32.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [9]:
# !pip install --upgrade pip
!pip install py-hanspell

  Using cached py-hanspell-1.1.tar.gz (3.0 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-4ib245rq/py-hanspell_51f45bf2a454430abd7b8fc6c744a918/setup.py", line 2, in <module>
          from pip.req import parse_requirements
      ModuleNotFoundError: No module named 'pip.req'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
!pip install WandB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 26.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8785 sha256=615c04facd5c423118720e7de4fcd44c4798f01963ef559bda311ba2434aacef
  Stored in directory: /opt/ml/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninsta

# Library

In [7]:
import os
import argparse
import pandas as pd
# !pip install git+https://github.com/jungin500/py-hanspell
from hanspell import spell_checker
import re

import transformers
import torch
import wandb
import torchmetrics
import pytorch_lightning as pl
import numpy as np


from sklearn.model_selection import KFold
from tqdm.auto import tqdm
from pytorch_lightning import seed_everything
from pytorch_lightning.loggers import WandbLogger

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Dataset

In [8]:
'''
Dataset
class DataLoader(pl.LightningDataModule)
		def setup(self, stage='fit')
'''
## 데이터셋을 객체로 만들어서 관리하는 class 입니다
## 수정 X
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets=[]):
        self.inputs = inputs
        self.targets = targets

    # 학습 및 추론 과정에서 데이터를 1개씩 꺼내오는 곳
    def __getitem__(self, idx):
        # 정답이 있다면 else문을, 없다면 if문을 수행합니다
        ## train 할때는 정답이 없을 수도 있으니까
        if len(self.targets) == 0:
            return torch.tensor(self.inputs[idx])
        else:
            return torch.tensor(self.inputs[idx]), torch.tensor(self.targets[idx])

    # 입력하는 개수만큼 데이터를 사용합니다
    def __len__(self):
        return len(self.inputs)

# Dataloader

## 일반 Dataloader

In [3]:
'''
Dataloader
main
'''
## dataset을 접근하기 편하게 만들어주는 class
class Dataloader(pl.LightningDataModule):
    ## 인자 및 차후에 사용할 객체들 선언
    def __init__(self, model_name, batch_size, shuffle, train_path, dev_path, test_path, predict_path):
        super().__init__()
        self.model_name = model_name
        self.batch_size = batch_size
        self.shuffle = shuffle

        self.train_path = train_path
        self.dev_path = dev_path
        self.test_path = test_path
        self.predict_path = predict_path

        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.predict_dataset = None

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, max_length=160)
        self.target_columns = ['label']
        self.delete_columns = ['id']
        self.text_columns = ['sentence_1', 'sentence_2']
        
        self.automatic_optimization = False

        # K-Fold
        self.k_fold = k_fold
        self.num_split = num_split
        self.k = k

        
    '''
    def setup(self, stage='fit'):
        def preprocessing(self, data):
	'''
    '''바꿔야할 tokenizing'''
    def tokenizing(self, dataframe):
        data = []
        for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
            # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
            text = '[SEP]'.join([item[text_column] for text_column in self.text_columns])
            outputs = self.tokenizer(text, add_special_tokens=True, padding='max_length', truncation=True)
            data.append(outputs['input_ids'])
        return data
    
    '''
    def setup(self, stage='fit'):
    '''
    '''바꿔야할 preprocessing'''
    def preprocessing(self, data):
        # 안쓰는 컬럼을 삭제합니다.
        data = data.drop(columns=self.delete_columns)

        # 타겟 데이터가 없으면 빈 배열을 리턴합니다.
        try:
            targets = data[self.target_columns].values.tolist()
        except:
            targets = []
        # 텍스트 데이터를 전처리합니다.
        inputs = self.tokenizing(data)

        return inputs, targets
    
    '''
	본체
    '''
    ## trainer fit이나 test 이전에 setup 코드가 먼저 돌아가게 되어있다
    ## fit에만 저렇게 stage에 들어간다
    ## 서재에다가 책 한권 넣는 느낌
    ## 수정 X
    def setup(self, stage='fit'):
        if stage == 'fit':
            # 학습 데이터와 검증 데이터셋을 호출합니다
            train_data = pd.read_csv(self.train_path)
            val_data = pd.read_csv(self.dev_path)

            # 학습데이터 준비
            train_inputs, train_targets = self.preprocessing(train_data)

            # 검증데이터 준비
            val_inputs, val_targets = self.preprocessing(val_data)

            # train 데이터만 shuffle을 적용해줍니다, 필요하다면 val, test 데이터에도 shuffle을 적용할 수 있습니다
            self.train_dataset = Dataset(train_inputs, train_targets)
            self.val_dataset = Dataset(val_inputs, val_targets)
        else:
            # 평가데이터 준비
            test_data = pd.read_csv(self.test_path)
            test_inputs, test_targets = self.preprocessing(test_data)
            self.test_dataset = Dataset(test_inputs, test_targets)

            predict_data = pd.read_csv(self.predict_path)
            predict_inputs, predict_targets = self.preprocessing(predict_data)
            self.predict_dataset = Dataset(predict_inputs, [])
    
    ## 예측 데이터 셋을 반환해주는 함수들이다
    ## 순서를 외우는 경우도 있기때문에 shuffle 해야한다
    ## 수정 X
    ####################################################################################################
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=args.shuffle)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset, batch_size=self.batch_size)
    ####################################################################################################

## KfoldDataLoader

In [9]:
'''
Dataloader
main
'''
class KfoldDataloader(pl.LightningDataModule):
    def __init__(self, model_name, batch_size, shuffle, \
                 train_path, dev_path, test_path, predict_path, \
                 ## 이게 K-fold를 위한 변수
                 # fold number
                 k: int = 1,  \
                 # split needs to be always the same for correct cross validation
                 split_seed: int = 12345, \
                 num_splits: int = 10
                 ):
        super().__init__()
        self.model_name = model_name
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.train_path = train_path
        self.dev_path = dev_path
        self.test_path = test_path
        self.predict_path = predict_path
        
        ## K-fold 위한 변수
        self.k = k
        self.split_seed = split_seed
        self.num_splits = num_splits

        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.predict_dataset = None

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, max_length=160)
        self.set_preprocessing()
    
        # 원본
        ## df = pd.DataFrame(data, columns=['source', 'sentence1','sentence2', 'label', 'binary_label'])
       
    '''
    def __init__()
	'''
    def set_preprocessing(self):
        # 데이터를 읽어서 타겟 컬럼, 안쓰는 컬럼, 텍스트 전처리가 필요한 컬럼명을 기록합니다.
        data = pd.read_csv(self.train_path)
        columns = data.columns
        
        self.target_columns = [columns[4]]

        self.delete_columns = [columns[0], columns[1]]
        self.text_columns = [columns[2], columns[3]]
        
    '''
    def setup(self, stage='fit'):
        def preprocessing(self, data):
	'''
    '''바꿔야할 tokenizing'''
    def tokenizing(self, dataframe):
        data = []
        for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
            # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
            text = '[SEP]'.join([item[text_column] for text_column in self.text_columns])
            outputs = self.tokenizer(text, add_special_tokens=True, padding='max_length', truncation=True)
            data.append(outputs['input_ids'])

        return data
    
    '''
    def setup(self, stage='fit'):
	'''
    '''바꿔야할 preprocessing'''
    def preprocessing(self, data):
        # 안쓰는 컬럼을 삭제합니다.
        data = data.drop(columns=self.delete_columns)

        # 타겟 데이터가 없으면 빈 배열을 리턴합니다.
        try:
            targets = data[self.target_columns].values.tolist()
        except:
            targets = []
        # 텍스트 데이터를 전처리합니다.
        inputs = self.tokenizing(data)

        return inputs, targets
    
    '''
    본체
    '''
    def setup(self, stage='fit'):
        train_data = pd.read_csv(self.train_path)
        val_data = pd.read_csv(self.dev_path)
        
        if stage == 'fit':
            # 데이터 준비
            ## 여기가 궁극적으로 다르다
            ############### 여기부터 ###############
            total_data = pd.concat([train_data, val_data], axis=0)
            total_input, total_targets = self.preprocessing(total_data)
            total_dataset = Dataset(total_input, total_targets)

            # 데이터셋 num_splits 번 fold
            kf = KFold(n_splits=self.num_splits, shuffle=self.shuffle, random_state=self.split_seed)
            all_splits = [k for k in kf.split(total_dataset)]
            
            # k번째 fold 된 데이터셋의 index 선택
            train_idxes, val_idxes = all_splits[self.k]
            train_idxes, val_idxes = train_idxes.tolist(), val_idxes.tolist()

            # fold한 index에 따라 데이터셋 분할
            self.train_dataset = [total_dataset[x] for x in train_idxes] 
            self.val_dataset = [total_dataset[x] for x in val_idxes]
            
            ############### 여기까지 ###############

        else:
            # 평가데이터 준비
            test_data = pd.read_csv(self.dev_path)
            predict_data = pd.read_csv(self.predict_path)
            
            test_inputs, test_targets = self.preprocessing(test_data)
            predict_inputs, X = self.preprocessing(predict_data)
            
            self.test_dataset = Dataset(test_inputs, test_targets)
            self.predict_dataset = Dataset(predict_inputs, [])
        
    
    ## 예측 데이터 셋을 반환해주는 함수들이다
    ## 순서를 외우는 경우도 있기때문에 shuffle 해야한다
    ## 수정 X
    ####################################################################################################
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=self.shuffle)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset, batch_size=self.batch_size)
    ####################################################################################################

## KfoldDataLoader team

In [12]:
class KfoldDataloader(pl.LightningDataModule):
    def __init__(self, model_name, batch_size, shuffle,
                 train_path, dev_path, test_path, predict_path,
                 ## 이게 K-fold를 위한 변수
                 # fold number
                 k,
                 k_fold=True,
                 num_split: int = 10,
                 # split needs to be always the same for correct cross validation
                 split_seed: int = 12345, 
                 seed = 11):
                 # k_fold=True, num_split=10, k=1, seed=11):
        super().__init__()
        # Base Info
        self.model_name = model_name
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed

        # Path
        self.train_path = train_path
        self.dev_path = dev_path
        self.test_path = test_path
        self.predict_path = predict_path

        # Dataset Init
        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.predict_dataset = None

        # Preproces
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            model_name, max_length=160)
        self.target_columns = ['label']
        self.delete_columns = ['id']
        self.text_columns = ['sentence_1', 'sentence_2']

        self.automatic_optimization = False

#         # K-Fold
        
#         self.k_fold = k_fold
#         self.num_split = num_split
#         self.k = k    
        
        ## K-fold 위한 변수
        self.k_fold = k_fold
        self.k = k
        self.split_seed = split_seed
        self.num_split = num_split
        
        self.set_preprocessing()
    
    def prepro_spell_checker(self, data):
        data[self.text_columns] = data[self.text_columns].applymap(
            lambda x : spell_checker.check(re.sub("&", " ", x)).checked)
        return data
        
    def set_preprocessing(self):
        # 데이터를 읽어서 타겟 컬럼, 안쓰는 컬럼, 텍스트 전처리가 필요한 컬럼명을 기록합니다.
        data = pd.read_csv(self.train_path)
        columns = data.columns
        
        self.target_columns = [columns[4]]

        self.delete_columns = [columns[0], columns[1]]
        self.text_columns = [columns[2], columns[3]]
    
    def tokenizing(self, df_input):
        data_input = []

        for idx, item in tqdm(df_input.iterrows(), desc='tokenizing', total=len(df_input)):
            # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
            text = '[SEP]'.join([item[text_column]
                                 for text_column in self.text_columns])
            outputs = self.tokenizer(
                text, add_special_tokens=True, padding='max_length', truncation=True, max_length=160)
            data_input.append(outputs['input_ids'])

        return data_input

    def preprocessing(self, data):
        # 안쓰는 컬럼을 삭제합니다.
        data = data.drop(columns=self.delete_columns)

        # 타겟 데이터가 없으면 빈 배열을 리턴합니다.
        try:
            targets = data[self.target_columns].values.tolist()
        except:
            targets = []
        # 텍스트 데이터를 전처리합니다.
        inputs = self.tokenizing(data)

        return inputs, targets

    def setup(self, stage='fit'):
        if stage == 'fit':
            # 학습 데이터와 검증 데이터셋을 호출합니다
            train_data = pd.read_csv(self.train_path)
            val_data = pd.read_csv(self.dev_path)

            # 검증데이터 준비
            #val_data = self.prepro_spell_checker(val_data)
            if self.k_fold:
                total_data = pd.concat([train_data, val_data], axis=0)
                total_input, total_targets = self.preprocessing(total_data)
                total_dataset = Dataset(total_input, total_targets)

                kf = KFold(n_splits=self.num_split, shuffle=True, random_state=self.seed)
                all_splits = [k for k in kf.split(total_dataset)]

                train_idx, val_idx = all_splits[self.k]
                train_idx, val_idx = train_idx.tolist(), val_idx.tolist()

                self.train_dataset = [total_dataset[i] for i in train_idx]
                self.val_dataset = [total_dataset[i] for i in val_idx]
            else:
                # 학습데이터 준비
                train_inputs, train_targets = self.set_preprocessing(train_data)

                # 검증데이터 준비
                val_inputs, val_targets = self.set_preprocessing(val_data)

                # train 데이터만 shuffle을 적용해줍니다, 필요하다면 val, test 데이터에도 shuffle을 적용할 수 있습니다
                self.train_dataset = Dataset(train_inputs, train_targets)
                self.val_dataset = Dataset(val_inputs, val_targets)
        else:
            # 평가데이터 준비
            test_data = pd.read_csv(self.test_path)
            #test_data = self.prepro_spell_checker(test_data)
            test_inputs, test_targets = self.preprocessing(test_data)
            self.test_dataset = Dataset(test_inputs, test_targets)

            predict_data = pd.read_csv(self.predict_path)
            #predict_data = self.prepro_spell_checker(predict_data)
            predict_inputs, predict_targets = self.preprocessing(predict_data)
            self.predict_dataset = Dataset(predict_inputs, [])

    def train_dataloader(self):
        def make_sampler(train_data):
            train_data['class'] = train_data['label'].apply(
                lambda x: int(x//1) if x != 5 else int(4))
            class_counts = train_data['class'].value_counts().to_list()
            # [3711, 1715, 1393, 1368, 1137]
            labels = train_data['class'].to_list()
            # [2, 4, 2, 3, 0, 2, 3, 0,
            num_samples = sum(class_counts)
            class_weights = [num_samples/class_counts[i]
                             for i in range(len(class_counts))]
            # [2.51, 5.436, 6.69, 6.81, 8.20]
            weights = [class_weights[labels[i]]
                       for i in range(int(num_samples))]
            # [6.69, 8.20, 6.69, 6.81, 2.51, 6.69
            sampler = torch.utils.data.WeightedRandomSampler(
                torch.DoubleTensor(weights), int(num_samples))
            return sampler

        # shuffle=args.shuffle
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=8)
                                            #, sampler=make_sampler(self.train_data))

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset, batch_size=self.batch_size)


# Model

In [13]:
class Model(pl.LightningModule):
    ## 함수들 인자 반환
    def __init__(self, model_name, lr):
        super().__init__()
        self.save_hyperparameters()

        self.model_name = model_name
        self.lr = lr

        # 사용할 모델을 호출합니다.
        '''바꿔야할 모델'''
        self.plm = transformers.AutoModelForSequenceClassification.from_pretrained(
            pretrained_model_name_or_path=model_name, num_labels=1)
        # Loss 계산을 위해 사용될 L1Loss를 호출합니다.
        '''바꿔야할 Loss function'''
        # self.loss_func = torch.nn.L1Loss()
        self.loss_func = torch.nn.MSELoss()
        ## 이건 Kfold 전용
        #self.loss_func = torch.nn.L1Loss()
        self.evaluation = torchmetrics.PearsonCorrCoef()
    
    ## 선학습 모델에 데이터를 입력하고 출력을 얻는 코드
    def forward(self, x):
        x = self.plm(x)['logits']

        return x
    
    ## 각각의 step에서는 이렇게 학습헤라
    ## 학습된 애는 시험을 볼 필요가 없으니까 val이나 test에서는 점수를 봐야하니까 봐야한다
    ## 수정 X
    ####################################################################################
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y.float())
        self.log("train_loss", loss)

        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y.float())
        self.log("val_loss", loss)
        
        self.log("val_pearson", torchmetrics.functional.pearson_corrcoef(logits.squeeze(), y.squeeze()))
        '''
        if self.bce:
            self.log("val_f1", self.evaluation(logits, y))
        else:
            self.log("val_pearson", self.evaluation(logits, y))
        '''
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)

        self.log("test_pearson", torchmetrics.functional.pearson_corrcoef(logits.squeeze(), y.squeeze()))
        '''
        if self.bce:
            self.log("test_f1", self.evaluation(logits, y))
        else:
            self.log("test_pearson", self.evaluation(logits, y))
        '''
        # return self.evaluation(logits, y)

    def predict_step(self, batch, batch_idx):
        x = batch
        logits = self(x)

        return logits.squeeze()
    ######################################################################################
    
    ## optimizer, LR scheduler를 선언하는 함수
    ## model의 train 이전에 이걸 실행하게 된다
    '''바꿔야할 optimizer'''
    # def configure_optimizers(self):
    #     optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
    #     return optimizer
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, total_iters=500)

        return (
            [optimizer],
            [
                {
                    'scheduler': scheduler,
                    'interval': 'step',
                    'frequency': 1,
                    'reduce_on_plateau': False,
                    'monitor': 'val_loss',
                }
            ]
        )

# __main__

In [16]:
# 하이퍼 파라미터 등 각종 설정값을 입력받습니다
# 터미널 실행 예시 : python3 run.py --batch_size=64 ...
# 실행 시 '--batch_size=64' 같은 인자를 입력하지 않으면 default 값이 기본으로 실행됩니다
## 여기 경로들을 잘 살펴서 봐야한다
## ./ : 현재 경로
parser = argparse.ArgumentParser()
'''바꿔야할 모델명'''
parser.add_argument('--model_name', default='klue/roberta-large', type=str)
# parser.add_argument('--model_name', default='beomi/KcELECTRA-base-v2022', type=str)

'''어디에 저장할꺼니'''
parser.add_argument('--entity_name', default='limha')
parser.add_argument('--project_name', default='STS_Lim')



'''수정해야할 hyperparameter'''
parser.add_argument('--train', default=True)
parser.add_argument('--sweeps_cnt', default=5)

parser.add_argument('--bce', default=False)

# parser.add_argument('--batch_size', default=4, type=int)
parser.add_argument('--batch_size', default=8, type=int)
parser.add_argument('--max_epoch', default=5, type=int)
parser.add_argument('--shuffle', default=True)
parser.add_argument('--lr', default=1.9555983936253056e-05, type=float)

## 단순히 경로
## 수정 X
parser.add_argument('--train_path', default="../data/train.csv")
parser.add_argument('--dev_path', default='../data/dev.csv')
parser.add_argument('--test_path', default='../data/dev.csv')
parser.add_argument('--predict_path', default='../data/test.csv')
args = parser.parse_args(args=[])
    
##수정 X
# dataloader와 model을 생성합니다.
'''
dataloader = Dataloader(args.model_name, args.batch_size, args.shuffle, args.train_path, args.dev_path,
                        args.test_path, args.predict_path)
model = Model(args.model_name, args.learning_rate)

# gpu가 없으면 accelerator='cpu', 있으면 accelerator='gpu'
trainer = pl.Trainer(accelerator='gpu', max_epochs=args.max_epoch, log_every_n_steps=1)

# Train part
trainer.fit(model=model, datamodule=dataloader)
trainer.test(model=model, datamodule=dataloader)

# 학습이 완료된 모델을 저장합니다.
torch.save(model, 'model.pt')
'''


## Kfold 완성..?
# 새 모델 생성

results = []
# K fold 횟수 3
nums_folds = 5
split_seed = 12345

# nums_folds는 fold의 개수, k는 k번째 fold datamodule

sum_predictions = np.zeros(1100)
for k in range(nums_folds):
    Kmodel = Model(args.model_name, args.lr)
    
    trainer = pl.Trainer(max_epochs=3)
    kfdataloader = KfoldDataloader(args.model_name, args.batch_size, args.shuffle,  
                                   args.train_path, args.dev_path, args.test_path, args.predict_path,
                                   k=k, split_seed=split_seed, num_split=nums_folds)
    kfdataloader.prepare_data()
    kfdataloader.setup()

    trainer.fit(model=Kmodel, datamodule=kfdataloader)
    trainer.test(model=Kmodel, datamodule=kfdataloader)
    
    predictions = trainer.predict(model=Kmodel, datamodule=kfdataloader) ## 여기서 predict step을 밟게 된다
    predictions = np.array(list(round(float(i), 1) for i in torch.cat(predictions)))
    
    sum_predictions += predictions


sum_predictions = (sum_predictions/5).tolist()

output = pd.read_csv('../data/sample_submission.csv')
output['target'] = sum_predictions
output.to_csv('output.csv', index=False)
    # results.extend(score)

# # 학습이 완료된 모델을 저장합니다.
# torch.save(Kmodel, 'model.pt')


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'cla

tokenizing:   0%|          | 0/9874 [00:00<?, ?it/s]

tokenizing:   0%|          | 0/9874 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                             | Params
----------------------------------------------------------------
0 | plm        | RobertaForSequenceClassification | 336 M 
1 | loss_func  | MSELoss                          | 0     
2 | evaluation | PearsonCorrCoef                  | 0     
----------------------------------------------------------------
336 M     Trainable params
0         Non-trainable params
336 M     Total params
1,346.630 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_pearson           0.958364725112915
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


tokenizing:   0%|          | 0/550 [00:00<?, ?it/s]

tokenizing:   0%|          | 0/1100 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

# 1) Inference

In [24]:
# Inference part
# 저장된 모델로 예측을 진행합니다.
model = torch.load('model.pt')
predictions = trainer.predict(model=model, datamodule=kfdataloader) ## 여기서 predict step을 밟게 된다

# 예측된 결과를 형식에 맞게 반올림하여 준비합니다.
predictions = list(round(float(i), 1) for i in torch.cat(predictions))

print(len(predictions))

# output 형식을 불러와서 예측된 결과로 바꿔주고, output.csv로 출력합니다.
output = pd.read_csv('../data/sample_submission.csv')
output['target'] = predictions
output.to_csv('output.csv', index=False)

tokenizing:   0%|          | 0/550 [00:00<?, ?it/s]

tokenizing:   0%|          | 0/1100 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

1100


# 2) WandB

In [26]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [27]:
!pip install wandb -qU

import wandb

wandb.login(relogin=True)

#wandb.login(key='cef73d59feda51d5dbe0cf219488ea4791940ed3', "relogin")


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /opt/ml/.netrc


True

### sweep_config(init)

In [7]:
sweep_config = {
    'method': 'random', # random: 임의의 값의 parameter 세트를 선택
                        # 'grid': 하나씩 모든 경우의 수 고려
                        # 'uniform': 균등분포에서 random하게 선택
                        # 'bayesian': 
    'parameters': {
        'lr':{
            'distribution': 'uniform',  # parameter를 설정하는 기준을 선택합니다. uniform은 연속적으로 균등한 값들을 선택합니다.
            'min':1e-5,                 # 최소값을 설정합니다.
            'max':1e-4                  # 최대값을 설정합니다.
        },
        'batch_size': {
        # integers between 16 and 64
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 16,
        'max': 64,
        }
      #   'max_epoch': {
      #   # integers between 2 and 4
      #   # with evenly-distributed logarithms 
      #   'distribution': 'q_log_uniform_values',
      #   'q': 1,
      #   'min': 2,
      #   'max': 4,
      # }
        
    }
}


sweep_config['metric'] = {'name':'val_pearson', 'goal':'maximize'}  # pearson 점수가 최대화가 되는 방향으로 학습을 진행합니다.

### sweep_config (team)

In [28]:
model_name = ['klue/roberta-small', 'klue/roberta-base', 'klue/bert-base', 'klue/electra-base'
                                  'KoELECTRA', 'KoBERT', 'klue/bert-base', 'klue/bert-large']

sweep_config = {'name': f"{args.entity_name}_based_lr-search-sweep",  # name : sweep_name
                'method': 'grid',  # 'grid', 'uniform', 'bayesian'
                    ## 만약에 4개 3개가 있으면 4번이 아니라 12번이다
                    ## sweep 자체의 early stopping
                    ## sweep count?
                'parameters': {
                    'model_name': {
                        'values': [
                                   'klue/roberta-small', \
                                   # 'klue/roberta-base', \
                                   # 'klue/roberta-large', \
                                   #'klue/bert-base', \
                                   #'klue/electra-base', \ X
                                   #'KoELECTRA', \ X
                                   # 'KoBERT', \
                                   #'klue/bert-large'
                                    ]
                    },
                        
                    'lr': {  # parameter  작성방식 여러개 있으니까, 노션 문서 참고
                        'values': [1e-5]
                    },
                    "batch_size": {
                        "values": [4]
                    },
                    "max_epoch": {
                        "values": [1]
                    },
                },
                # goal : maximize, minimize
                'metric': {'name': 'val_pearson', 'goal': 'maximize'}
                }

### sweep_train

In [29]:
def sweep_train(config=None):
    wandb.init(config=config)
    config = wandb.config
    
    # dataloader = Dataloader(config.model_name, config.batch_size, args.shuffle, args.train_path, args.dev_path, args.test_path, args.predict_path)
    dataloader = KfoldDataloader(config.model_name, config.batch_size, args.shuffle, args.train_path, args.dev_path, args.test_path, args.predict_path)

    
    wandb_logger = WandbLogger(project="klue-test0") #project= 현재위치에 해당이름의 폴더가 생성됨
    ## ??: 밑에 project 부분과 겹치는건가?
   
    
    for k in range(nums_folds):
        model = Model(args.model_name, config.lr)
        trainer = pl.Trainer(accelerator='gpu', max_epochs=config.max_epoch, logger=wandb_logger, log_every_n_steps=1)
        
        kfdataloader = KfoldDataloader(args.model_name, args.batch_size, args.shuffle,  
                                       args.train_path, args.dev_path, args.test_path, args.predict_path,
                                       k=k, split_seed=split_seed, num_splits=nums_folds)
        kfdataloader.prepare_data()
        kfdataloader.setup()

        trainer.fit(model=Kmodel, datamodule=kfdataloader)
        trainer.test(model=Kmodel, datamodule=kfdataloader)
    # trainer.fit(model=model, datamodule=dataloader)
    # trainer.test(model=model, datamodule=dataloader)
   
    # torch.save(model, f'model{config.batch_size}-{config.lr}-{config.max_epoch}.pt')

In [30]:
# Sweep 생성

sweep_id = wandb.sweep(
    sweep=sweep_config,     # config 딕셔너리를 추가합니다.
    project=args.project_name, entity = args.entity_name  # project의 이름을 추가합니다. wandb에 올라가는 project 이름
)

wandb.agent(
    sweep_id=sweep_id,      # sweep의 정보를 입력하고
    function=sweep_train,   # train이라는 모델을 학습하는 코드를
    # count=3                 # 총 3회 실행해봅니다.
)

Create sweep with ID: ow410yta
Sweep URL: https://wandb.ai/limha/STS_Lim/sweeps/ow410yta


wandb: Agent Starting Run: d60qd6rn with config:
wandb: 	batch_size: 4
wandb: 	lr: 1e-05
wandb: 	max_epoch: 1
wandb: 	model_name: klue/roberta-small
wandb: Currently logged in as: limha. Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'class

tokenizing:   0%|          | 0/9874 [00:00<?, ?it/s]

tokenizing:   0%|          | 0/9874 [00:00<?, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'model_name' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_pearson          0.7781239151954651
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
test_pearson,▁▁▁
train_loss,▇▆▆▇█▇▅▅▃▇▆▆▆▆▃▆▃▅▅▃▂▃▄▄▄▄▄▄▄▂▄▃▃▃▄▂▁▄▄▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▁
epoch,1
test_pearson,0.77812
train_loss,0.34879
trainer/global_step,1646
val_loss,0.59385
val_pearson,nan


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
